this notebook is for the capstone assignement

In [1]:
import numpy as np
import pandas as pd

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [10]:
# make a function to check missing data

def check_missing_data(df_name):
    missing_data=df_name.isnull()

    for column in missing_data.columns.values.tolist():
        print(column)
        print (missing_data[column].value_counts())
        print("")   

# check missing data in the original list
check_missing_data(df)

SEVERITYCODE
False    194673
Name: SEVERITYCODE, dtype: int64

X
False    189339
True       5334
Name: X, dtype: int64

Y
False    189339
True       5334
Name: Y, dtype: int64

OBJECTID
False    194673
Name: OBJECTID, dtype: int64

INCKEY
False    194673
Name: INCKEY, dtype: int64

COLDETKEY
False    194673
Name: COLDETKEY, dtype: int64

REPORTNO
False    194673
Name: REPORTNO, dtype: int64

STATUS
False    194673
Name: STATUS, dtype: int64

ADDRTYPE
False    192747
True       1926
Name: ADDRTYPE, dtype: int64

INTKEY
True     129603
False     65070
Name: INTKEY, dtype: int64

LOCATION
False    191996
True       2677
Name: LOCATION, dtype: int64

EXCEPTRSNCODE
True     109862
False     84811
Name: EXCEPTRSNCODE, dtype: int64

EXCEPTRSNDESC
True     189035
False      5638
Name: EXCEPTRSNDESC, dtype: int64

SEVERITYCODE.1
False    194673
Name: SEVERITYCODE.1, dtype: int64

SEVERITYDESC
False    194673
Name: SEVERITYDESC, dtype: int64

COLLISIONTYPE
False    189769
True       4904
Name: C

In [159]:
# drop unnecessary columns

df_clean=df.drop(['X','Y','OBJECTID','INCKEY','COLDETKEY','REPORTNO','STATUS','INTKEY','LOCATION','EXCEPTRSNCODE','EXCEPTRSNDESC','SEVERITYCODE.1','SEVERITYDESC','INCDATE','SDOT_COLDESC','SDOTCOLNUM','ST_COLDESC','SEGLANEKEY','CROSSWALKKEY'],axis=1)
df_clean.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR
0,2,Intersection,Angles,2,0,0,2,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,NaN,N,Overcast,Wet,Daylight,NaN,NaN,10,N
1,1,Block,Sideswipe,2,0,0,2,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,NaN,11,N
2,1,Block,Parked Car,4,0,0,3,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,NaN,0,Overcast,Dry,Daylight,NaN,NaN,32,N
3,1,Block,Other,3,0,0,3,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,NaN,N,Clear,Dry,Daylight,NaN,NaN,23,N
4,2,Intersection,Angles,2,0,0,2,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,NaN,0,Raining,Wet,Daylight,NaN,NaN,10,N


In [30]:
# check missig data again

# check_missing_data(df_clean)

In [160]:
# fill empty with 'N' in below columns
df_clean.fillna({'INATTENTIONIND':'N','PEDROWNOTGRNT':'N','SPEEDING':'N'},inplace=True)


# replace 'Y/N' into '1/0' in column 'UNDERINFL'
df_clean.UNDERINFL.replace('Y','1',inplace=True)
df_clean.UNDERINFL.replace('N','0',inplace=True)

df_clean.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR
0,2,Intersection,Angles,2,0,0,2,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,N,0,Overcast,Wet,Daylight,N,N,10,N
1,1,Block,Sideswipe,2,0,0,2,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,N,0,Raining,Wet,Dark - Street Lights On,N,N,11,N
2,1,Block,Parked Car,4,0,0,3,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,N,0,Overcast,Dry,Daylight,N,N,32,N
3,1,Block,Other,3,0,0,3,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,N,0,Clear,Dry,Daylight,N,N,23,N
4,2,Intersection,Angles,2,0,0,2,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,N,0,Raining,Wet,Daylight,N,N,10,N


In [161]:
# drop rows with empty value in any column

df_clean=df_clean.dropna()
df_clean.shape

(182895, 19)

In [162]:
# data standardzation
# this takes some time

df_clean['DATETIME']=pd.to_datetime(df_clean.INCDTTM)
df_clean.drop(['INCDTTM'],axis=1,inplace=True)

cols=['INATTENTIONIND','PEDROWNOTGRNT','SPEEDING']
df_clean[cols]=df_clean[cols].replace('Y','1')
df_clean[cols]=df_clean[cols].replace('N','0')
df_clean.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,DATETIME
0,2,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Overcast,Wet,Daylight,0,0,10,N,2013-03-27 14:54:00
1,1,Block,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),16,0,0,Raining,Wet,Dark - Street Lights On,0,0,11,N,2006-12-20 18:55:00
2,1,Block,Parked Car,4,0,0,3,Mid-Block (not related to intersection),14,0,0,Overcast,Dry,Daylight,0,0,32,N,2004-11-18 10:20:00
3,1,Block,Other,3,0,0,3,Mid-Block (not related to intersection),11,0,0,Clear,Dry,Daylight,0,0,23,N,2013-03-29 09:26:00
4,2,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Raining,Wet,Daylight,0,0,10,N,2004-01-28 08:04:00


In [163]:
# cut the latest 30000 samples from dataset
df_clean.sort_values(by=['DATETIME'],ascending=False,inplace=True)
df_clean=df_clean[0:30000]
df_clean.reset_index(inplace=True)
df_clean.drop('index',axis=1, inplace=True)
df_clean.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,DATETIME
0,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Clear,Dry,Daylight,0,0,10,N,2020-04-28 11:29:00
1,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Clear,Dry,Daylight,0,0,10,N,2020-04-27 13:38:00
2,1,Block,Other,1,0,0,1,Mid-Block (not related to intersection),26,0,0,Clear,Dry,Dawn,0,0,50,N,2020-04-27 05:28:00
3,2,Intersection,Left Turn,3,0,0,2,At Intersection (intersection related),11,0,0,Overcast,Dry,Daylight,0,0,28,N,2020-04-26 15:20:00
4,1,Block,Parked Car,2,0,0,2,Mid-Block (not related to intersection),11,0,0,Overcast,Wet,Dark - Street Lights On,0,0,32,N,2020-04-25 23:00:00


In [172]:
# ramdonly drop 10000 samples of category-1
df_low=df_clean[df_clean.SEVERITYCODE==1]
drop_index=np.random.choice(df_low.index,size=10000)
df_clean.drop(drop_index,inplace=True)

In [173]:
# well, not exactly 10000 was dropped, but still works
df_clean.SEVERITYCODE.value_counts()

1    12104
2    10017
Name: SEVERITYCODE, dtype: int64

In [174]:
df_clean.head(50)

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,DATETIME
0,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Clear,Dry,Daylight,0,0,10,N,2020-04-28 11:29:00
2,1,Block,Other,1,0,0,1,Mid-Block (not related to intersection),26,0,0,Clear,Dry,Dawn,0,0,50,N,2020-04-27 05:28:00
3,2,Intersection,Left Turn,3,0,0,2,At Intersection (intersection related),11,0,0,Overcast,Dry,Daylight,0,0,28,N,2020-04-26 15:20:00
5,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Raining,Wet,Daylight,0,0,10,N,2020-04-25 08:29:00
6,1,Block,Parked Car,2,0,0,2,Mid-Block (not related to intersection),14,0,0,Clear,Dry,Daylight,0,0,32,Y,2020-04-24 15:30:00
9,2,Intersection,Rear Ended,3,0,0,2,At Intersection (intersection related),14,0,0,Clear,Dry,Daylight,0,0,14,N,2020-04-24 13:05:00
10,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Clear,Dry,Daylight,0,0,10,N,2020-04-24 12:43:00
12,2,Block,Other,1,0,0,1,Mid-Block (not related to intersection),26,0,0,Clear,Dry,Daylight,0,1,57,N,2020-04-24 10:30:00
15,2,Intersection,Cycles,2,0,1,1,At Intersection (intersection related),18,0,0,Raining,Wet,Daylight,0,0,5,N,2020-04-23 17:09:00
16,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Clear,Dry,Daylight,0,0,10,N,2020-04-23 12:34:00


In [171]:
df_clean.iloc[14100]

SEVERITYCODE                                            1
ADDRTYPE                                            Block
COLLISIONTYPE                                       Other
PERSONCOUNT                                             3
PEDCOUNT                                                0
PEDCYLCOUNT                                             0
VEHCOUNT                                                3
JUNCTIONTYPE      Mid-Block (not related to intersection)
SDOT_COLCODE                                           11
INATTENTIONIND                                          1
UNDERINFL                                               0
WEATHER                                             Clear
ROADCOND                                              Dry
LIGHTCOND                                        Daylight
PEDROWNOTGRNT                                           0
SPEEDING                                                0
ST_COLCODE                                             23
HITPARKEDCAR  